In [4]:
# Plot Surrogate Predictions
from gpPredict import *
import pandas as pd
import numpy as np
from diff_evolution_col import *
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
import os

import pickle
filename = 'logistic_regression_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))


def predict_with_uncertainty(new_vector, model, n_samples=1):
    # Predict the class
    prediction = model.predict(new_vector)
    # Predict the probabilities
    probabilities = model.predict_proba(new_vector)
    # Generate n_samples samples
    samples = np.random.choice([0, 1], size=(n_samples, 1), p=probabilities[0])
    return prediction, probabilities[0][prediction[0]], samples


def plot_hysteresis(hyst_data, style):
    
    plt.plot(hyst_data["disp"], hyst_data["force"], **style)
    plt.xlabel("Displacement")
    plt.ylabel("Force")

    pass

In [5]:
ii = 0

cwd = os.getcwd()

# ------
# Here, call surrogate model for flexure failure
# ------
f_surrogate_dir = os.path.join(cwd, 'quoFEM_Surrogate', 'FlexureAllTypes')
f_template_dir = os.path.join(f_surrogate_dir, 'templatedir_SIM')
f_surrogate_file = os.path.join(f_surrogate_dir, 'SimGpModel.json')

# Load the test and train data files
f_test_data = pd.read_csv(os.path.join(f_surrogate_dir, 'test_data.csv'))
f_test_data['DataType'] = 'Test'

f_train_data = pd.read_csv(os.path.join(f_surrogate_dir, 'train_data.csv'))
f_train_data['DataType'] = 'Train'

# ------
# Here, call surrogate model for shear failure mode
# ------
s_surrogate_dir = os.path.join(cwd, 'quoFEM_Surrogate', 'ShearAllTypes')
s_template_dir = os.path.join(f_surrogate_dir, 'templatedir_SIM')
s_surrogate_file = os.path.join(s_surrogate_dir, 'SimGpModel.json')

# load the merged_data.csv file
s_test_data = pd.read_csv(os.path.join(s_surrogate_dir, 'test_data.csv'))
s_test_data['DataType'] = 'Test'

s_train_data = pd.read_csv(os.path.join(s_surrogate_dir, 'train_data.csv'))
s_train_data['DataType'] = 'Train'


# Combine the training and testing data
merged_data = pd.concat([f_train_data, f_test_data, s_train_data, s_test_data])

# Restart indices in merged_data
merged_data.reset_index(drop=True, inplace=True)

# Input_json is the same for both cases
input_json = os.path.join(cwd, 'quoFEM_Surrogate', 'scInput.json')

cal_params_index = np.arange(0, 14)
nd_params_index = np.arange(18, 24)

# Extract the calibrated and nondimensional parameters for both training and testing
cal_params_all = merged_data.iloc[:, cal_params_index]
nondim_params_all = merged_data.iloc[:, nd_params_index]

# Save the merged data
#merged_data.to_csv(r'C:\Users\Miguel.MIGUEL-DESK\Documents\GitHub\RC_Column_Model\merged_calibration_data.csv', index=False)
merged_data.columns


Index(['eta1', 'kappa_k', 'kappa', 'sig', 'lam', 'mup', 'sigp', 'rsmax', 'n',
       'alpha', 'alpha1', 'alpha2', 'betam1', 'gamma', 'UniqueId', 'Name',
       'PeakDrift', 'FailureType', 'ar', 'lrr', 'srr', 'alr', 'sdr', 'smr',
       'Type', 'DataType'],
      dtype='object')

In [ ]:
# Find where the input data is
cwd = os.getcwd()
calsDir = 'cals_dr_005'

# Define columns for the performance data
columns = ['UniqueId', 'Name', 'TestOrTrain', 'Type', 'RMSECal', 'RMSESurr', 'EnergyCal', 'EnergySurr', 'StiffCal', 'StiffSur', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']
surrogate_performance_data = pd.DataFrame(columns=columns)

for ii in range(0, len(merged_data)):

    # Define file name
    file = 'test_' + str(merged_data['UniqueId'][ii]).zfill(3) + '.json'
    with open(os.path.join(cwd, calsDir, file), 'r') as f:
        test_data = json.load(f)
    
    # Load displacement and force data
    disp = test_data['data']['disp']
    force = test_data['data']['force']

    # Load test name
    name = test_data['Name']
    name2 = merged_data['Name'][ii]
    # print(ii, file, name, )

    # Define data_type test or train
    dataType = merged_data['DataType'][ii]

    # Extract the calibrated and nondimensional parameters for both training and testing
    calParams = merged_data.iloc[ii, cal_params_index].to_list()
    ndParams = merged_data.iloc[ii, nd_params_index]
    
    params_list = [
        ["RV_column1", ndParams.iloc[0]],
        ["RV_column2", ndParams.iloc[1]],
        ["RV_column3", ndParams.iloc[2]],
        ["RV_column4", ndParams.iloc[3]],
        ["RV_column5", ndParams.iloc[4]], 
        ["RV_column6", ndParams.iloc[5]]
        ]
    
    #print(calParams)
    #print(ndParams)
    
    # Here, call surrogate model
    # Select the surrogate model
    '''if test_data['FailureType'] == 'Flexure':
        print('Flexure-type failure')
        surrogate_file = f_surrogate_file
    else:
        print('Shear-type failure')
        surrogate_file = s_surrogate_file'''
    
    # Predict failure type with the logistic regression model
    new_vector = new_vector = pd.DataFrame(np.array(ndParams).reshape(1, -1), columns=['ar', 'lrr', 'srr', 'alr', 'sdr', 'smr'])

    # Try the function
    prediction, probability, samples = predict_with_uncertainty(new_vector, loaded_model)

    # If prediction == 1 then it is a flexure failure
    if prediction == 1:
        print('Flexure-type failure with probability: ', probability)
        surrogate_file = f_surrogate_file
    else:
        print('Shear-type failure with probability: ', 1 - probability)
        surrogate_file = s_surrogate_file

    print(f"Actual failure mode (PEER): {test_data['FailureType']}")

    params = main(params_list, [], surrogate_file, 'dummyout.out', input_json)
    
        
    gp_predicted = []
        
    for vals in params[0]:
        gp_predicted.append(vals)
    
    print(gp_predicted)
    print(calParams)
    # Run the model with calibrated parameters
    model_data_cal, exp_data = run_model(calParams, test_data)

    # Run the model with surrogate predicted parameters
    model_data_sur, exp_data = run_model(gp_predicted, test_data)

    plt.figure(figsize=(10, 8))
    plt.subplot(2, 2, 1)
    plot_hysteresis(model_data_cal, {'color': 'red', 'linestyle': '--', 'label': 'Calibrated', 'linewidth': 0.75})
    plot_hysteresis(exp_data, {'color': 'black', 'linestyle': '--', 'label': 'Experimental', 'linewidth': 0.75})
    plot_hysteresis(model_data_sur, {'color': 'blue', 'linestyle': ':', 'label': 'Surrogate'})
    plt.grid()
    plt.legend()
    #plt.show()

    
    # Compute the residuals betweeen the experimental and model data
    residuals_cal = (exp_data['force'] - model_data_cal['force'])
    residuals_sur = (exp_data['force'] - model_data_sur['force'])
    rmse_cal = np.sqrt(np.mean(residuals_cal**2) / len(residuals_cal))
    rmse_sur = np.sqrt(np.mean(residuals_sur**2) / len(residuals_sur))
    print(f'RMSE Calibrated: {rmse_cal}, RMSE Surrogate: {rmse_sur}')
    
    # Calculate residuals with respect to calibrated curve instead of experimental data
    '''residuals_cal = (exp_data['force'] - model_data_cal['force'])        # keep the same
    residuals_sur = (model_data_cal['force'] - model_data_sur['force'])  # Surrogate residuals are with respect to calibrated curve
    rmse_cal = np.sqrt(np.mean(residuals_cal**2) / len(residuals_cal))   # keep the same
    rmse_sur = np.sqrt(np.mean(residuals_sur**2) / len(residuals_sur))   # Surrogate residuals are with respect to calibrated curve
    print(f'RMSE Calibrated: {rmse_cal}, RMSE Surrogate: {rmse_sur}')
'''

    # Plot the residuals
    plt.subplot(2, 2, 2)
    sns.histplot(data=residuals_cal, bins=20, color='red', label='Calibrated Residuals', kde=True, alpha=0.5)
    sns.histplot(data=residuals_sur, bins=20, color='blue', label='Surrogate Residuals', kde=True, alpha=0.5)
    # plt.plot(exp_data['disp'], residuals_sur, 'b.', label='Surrogate Residuals')


    # Compute the cumulative energy dissipated
    energy_cal = sp.integrate.cumulative_trapezoid(model_data_cal['force'], exp_data['disp'], initial=0)
    energy_sur = sp.integrate.cumulative_trapezoid(model_data_sur['force'], exp_data['disp'], initial=0)
    energy_exp = sp.integrate.cumulative_trapezoid(exp_data['force'], exp_data['disp'], initial=0)

    energy_abs_cal = np.sum(np.sqrt((energy_cal - energy_exp)**2) / len(energy_exp))
    energy_abs_sur = np.sum(np.sqrt((energy_sur - energy_cal)**2) / len(energy_cal))  # Surrogate residuals are with respect to calibrated curve

    plt.subplot(2, 2, 3)
    plt.plot(exp_data['disp'], energy_exp, 'k--', label='Experimental Energy')
    plt.plot(exp_data['disp'], energy_cal, 'r--', label='Calibrated Energy')
    plt.plot(exp_data['disp'], energy_sur, 'b-', label='Surrogate Energy')
    plt.grid()

    # Compute the tangent stiffness
    tangent_stiffness_cal = np.gradient(model_data_cal['force'], model_data_cal['disp'])
    tangent_stiffness_sur = np.gradient(model_data_sur['force'], model_data_sur['disp'])
    tangent_stiffness_exp = np.gradient(exp_data['force'], exp_data['disp'])
    
    # Add trendlines for each
    stiff_ratio_cal = tangent_stiffness_cal / tangent_stiffness_exp
    stiff_ratio_sur = tangent_stiffness_sur / tangent_stiffness_cal

    # Plot mean
    mean_cal = np.median(stiff_ratio_cal[np.isfinite(stiff_ratio_cal)])
    mean_sur = np.median(stiff_ratio_sur[np.isfinite(stiff_ratio_sur)])

    plt.subplot(2, 2, 4)
    plt.plot(exp_data['disp'], tangent_stiffness_cal/tangent_stiffness_exp, '.', color='red', label='Calibrated Tangent Stiffness', alpha=0.5)
    plt.plot(exp_data['disp'], tangent_stiffness_sur/tangent_stiffness_exp, '.', color='blue', label='Surrogate Tangent Stiffness', alpha=0.5)
    plt.axhline(mean_cal, color='red', linestyle='--', label='Mean Calibrated')
    plt.axhline(mean_sur, color='blue', linestyle='--', label='Mean Surrogate')

    #plt.plot(exp_data['disp'], tangent_stiffness_cal/tangent_stiffness_exp, 'r.', label='Calibrated Tangent Stiffness')
    #plt.plot(exp_data['disp'], tangent_stiffness_sur/tangent_stiffness_exp, 'b.', label='Surrogate Tangent Stiffness')
    plt.ylim(-2.0, 3.0)
    plt.show()
    # cc += 1

    # Create a dataframe for the performance data of this single case, includin the nondimensional parameters
    performance_data = pd.DataFrame([[str(merged_data['UniqueId'][ii]).zfill(3), name, dataType, test_data['FailureType'], rmse_cal, rmse_sur, energy_abs_cal, energy_abs_sur, mean_cal, mean_sur] + list(ndParams)], columns=columns)

    # Append to the main dataframe
    surrogate_performance_data = pd.concat([surrogate_performance_data, performance_data], ignore_index=True) 

# Save the performance data to a CSV file
surrogate_performance_data.to_csv('surrogate_performance_data_new.csv', index=False)
    

0 test_112.json Galeota et al. 1996, AA4
Flexure-type failure with probability:  0.905546661416196
Actual failure mode (PEER): Flexure
RV_column1
[[4.56]]
RV_column2
[[0.08105309]]
RV_column3
[[0.02881888]]
RV_column4
[[0.2]]
RV_column5
[[3.125]]
RV_column6
[[0.47854754]]
[1.43001988e+00 1.02465994e+00 1.00250727e+00 4.61934700e-01
 5.14616098e-01 3.84720656e+00 5.18505992e+00 6.56836638e-01
 5.08143091e+00 2.01985356e-03 4.09728614e+00 8.48474370e-01
 1.95858148e-02 4.88337612e-01]
[1.4300198758984863, 1.024659935305131, 1.0025072733981117, 0.4619347004858803, 0.5146160980072871, 3.8472065633769534, 5.185059918476723, 0.6568366375646059, 5.081430913938734, 0.002019853555505246, 4.097286138859367, 0.8484743703787365, 0.01958581480079128, 0.48833761234721845]
[1.2407890826356311, 1.042357091077159, 0.9931238291441294, 0.4700719738675702, 0.513122708852944, 4.182510954902258, 6.537995745987935, 0.8922253202664467, 5.247176602424152, 3.056553433567616e-05, 3.338790469633465, 1.80325017558

KeyboardInterrupt: 

In [ ]:
surrogate_performance_data

,UniqueId,Name,TestOrTrain,Type,RMSECal,RMSESurr,EnergyCal,EnergySurr,StiffCal,StiffSur,a1,a2,a3,a4,a5,a6
0,112,"Galeota et al. 1996, AA4",Train,Flexure,0.000794,0.000962,0.001511,0.004346,0.961578,1.044139,4.560000,0.081053,0.028819,0.200000,3.125000,0.478548
1,060,"Muguruma et al. 1989, BL-1",Train,Flexure,0.000981,0.001160,0.005189,0.014401,0.914078,1.031358,2.500000,0.131140,0.045819,0.253886,0.972222,0.736114
2,216,"Paultre and Legeron, 2000, No. 1006025",Train,Flexure,0.000847,0.001332,0.019765,0.011450,0.947581,0.918685,6.557377,0.118369,0.078085,0.276522,0.884956,0.229786
3,299,"Petrovski and Ristic 1984, M1E1",Train,Flexure,0.001857,0.001756,0.023836,0.022301,0.710297,1.064267,2.000000,0.410811,0.053067,0.389930,1.000000,0.928720
4,156,"Nosho et al. 1996, No. 1",Train,Flexure,0.001284,0.001298,0.002626,0.003151,1.121052,0.892726,7.637795,0.101990,0.008439,0.339495,6.047619,0.451830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,283,"Ang et al. 1985, No. 17",Test,Flexure-Shear,0.001515,0.001308,0.084482,0.016750,1.014900,0.896846,2.000000,0.417725,0.022998,0.100067,1.666667,1.336156
287,080,"Wight and Sozen 1973, No. 25.033(East)",Test,Shear,0.000976,0.001121,0.009035,0.008407,0.988759,1.143760,2.872131,0.361124,0.033161,0.071259,3.359788,1.093090
288,282,"Ang et al. 1985, No. 16",Test,Shear,0.001101,0.001980,0.016937,0.038231,0.940156,1.034520,2.000000,0.240552,0.022072,0.000000,1.666667,0.848960
289,281,"Ang et al. 1985, No. 15",Test,Flexure-Shear,0.001130,0.001713,0.017495,0.028776,1.025680,0.723038,2.000000,0.407644,0.022793,0.000000,1.666667,1.406019
